In [1]:
# !pip uninstall mlops-ai

In [2]:
import sys, os
sys.path.append('../..')

from mlops.tracking import set_active_experiment, set_active_project, create_project, create_experiment
from mlops.tracking import start_iteration
from mlops.monitoring import get_model_by_name, send_prediction

## Setup Iteration with target model for monitoring 

In [3]:
project = create_project(
    title="Project for monitoring", 
    description="Showcase project for monitoring :)",
    status='in_progress'
)
set_active_project(project_id=project['_id'])

'Active project set to: 6556605c8621432f52372b98'

In [4]:
experiment = create_experiment(
    name="Iris modeling",
    description="Iris classification models"
)
set_active_experiment(experiment_id=experiment['id'])

'Active experiment set to: 6556605c8621432f52372b99'

In [5]:
import pandas as pd 
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

url = 'https://raw.githubusercontent.com/TripathiAshutosh/dataset/main/iris.csv'
df = pd.read_csv(filepath_or_buffer=url, sep=',')
y = LabelEncoder().fit_transform(df['class'])
X = df.drop(columns=['class'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify = y, random_state=42)

parameters = {'n_estimators': 100, 'max_depth': 5}
model = RandomForestClassifier(**parameters)
model.fit(X_train, y_train)
prediction = model.predict(X_test)
metrics = {
    'accuracy': round(accuracy_score(y_test, prediction), 3),
    'precision': round(precision_score(y_test, prediction, average='macro'), 3),
    'recall': round(recall_score(y_test, prediction, average='macro'), 3),
    'f1': round(f1_score(y_test, prediction, average='macro'), 3)
}

filename='../test_files/rf_iris.pkl'
with open(filename, "wb") as f:
    pickle.dump(model, f)

In [6]:
with start_iteration(iteration_name='RF model v1') as iteration:
    iteration.log_parameters(parameters=parameters)
    iteration.log_metrics(metrics=metrics)
    iteration.log_path_to_model(path_to_model=os.path.abspath(filename))

## Valid prediction

In [7]:
X_test

,sepal-length,sepal-width,petal-length,petal-width
21,5.1,3.7,1.5,0.4
7,5.0,3.4,1.5,0.2
35,5.0,3.2,1.2,0.2
23,5.1,3.3,1.7,0.5
132,6.4,2.8,5.6,2.2
...,...,...,...,...
53,5.5,2.3,4.0,1.3
22,4.6,3.6,1.0,0.2
102,7.1,3.0,5.9,2.1
136,6.3,3.4,5.6,2.4


In [8]:
send_prediction(
    model_name='Demo presentation',
    data=X_test
)

[{'id': '655660868621432f52372b9c',
  'prediction_date': '2023-11-16T19:33:42.112565',
  'input_data': {'sepal-length': 5.1,
   'sepal-width': 3.7,
   'petal-length': 1.5,
   'petal-width': 0.4},
  'prediction': 0.0,
  'actual': None},
 {'id': '655660868621432f52372b9d',
  'prediction_date': '2023-11-16T19:33:42.118564',
  'input_data': {'sepal-length': 5.0,
   'sepal-width': 3.4,
   'petal-length': 1.5,
   'petal-width': 0.2},
  'prediction': 0.0,
  'actual': None},
 {'id': '655660868621432f52372b9e',
  'prediction_date': '2023-11-16T19:33:42.125563',
  'input_data': {'sepal-length': 5.0,
   'sepal-width': 3.2,
   'petal-length': 1.2,
   'petal-width': 0.2},
  'prediction': 0.0,
  'actual': None},
 {'id': '655660868621432f52372b9f',
  'prediction_date': '2023-11-16T19:33:42.132075',
  'input_data': {'sepal-length': 5.1,
   'sepal-width': 3.3,
   'petal-length': 1.7,
   'petal-width': 0.5},
  'prediction': 0.0,
  'actual': None},
 {'id': '655660868621432f52372ba0',
  'prediction_date':

In [10]:
pd.DataFrame(get_model_by_name(model_name='Demo presentation')['predictions_data'])

,id,prediction_date,input_data,prediction,actual
0,655660868621432f52372b9c,2023-11-16T19:33:42.112000,"{'sepal-length': 5.1, 'sepal-width': 3.7, 'pet...",0.0,NaN
1,655660868621432f52372b9d,2023-11-16T19:33:42.118000,"{'sepal-length': 5.0, 'sepal-width': 3.4, 'pet...",0.0,NaN
2,655660868621432f52372b9e,2023-11-16T19:33:42.125000,"{'sepal-length': 5.0, 'sepal-width': 3.2, 'pet...",0.0,NaN
3,655660868621432f52372b9f,2023-11-16T19:33:42.132000,"{'sepal-length': 5.1, 'sepal-width': 3.3, 'pet...",0.0,NaN
4,655660868621432f52372ba0,2023-11-16T19:33:42.139000,"{'sepal-length': 6.4, 'sepal-width': 2.8, 'pet...",2.0,NaN
...,...,...,...,...,...
70,655660868621432f52372be2,2023-11-16T19:33:42.712000,"{'sepal-length': 5.5, 'sepal-width': 2.3, 'pet...",1.0,1.0
71,655660868621432f52372be3,2023-11-16T19:33:42.723000,"{'sepal-length': 4.6, 'sepal-width': 3.6, 'pet...",0.0,0.0
72,655660868621432f52372be4,2023-11-16T19:33:42.734000,"{'sepal-length': 7.1, 'sepal-width': 3.0, 'pet...",2.0,2.0
73,655660868621432f52372be5,2023-11-16T19:33:42.746000,"{'sepal-length': 6.3, 'sepal-width': 3.4, 'pet...",2.0,2.0


## Invalid prediction (wrong column names)

In [11]:
send_prediction(
    model_name='Demo presentation',
    data=pd.DataFrame({'X1': [121], 'X2': [3]})
)

Exception: Request failed with status code 400: Cannot make prediction: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- X1
- X2
Feature names seen at fit time, yet now missing:
- petal-length
- petal-width
- sepal-length
- sepal-width


## Invalid prediction (wrong value type)

In [12]:
send_prediction(
    model_name='Demo presentation',
    data=pd.DataFrame({
        'sepal-length': ['WRONG value'],
        'sepal-width': [2.8],
        'petal-length': [4.9],
        'petal-width': [2.0]
    })
)

Exception: Request failed with status code 400: Cannot make prediction: could not convert string to float: 'WRONG value'

## Invalid + valid together

In [15]:
send_prediction(
    model_name='Demo presentation',
    data=pd.DataFrame({
        'sepal-length': [1.0, 'WRONG value'],
        'sepal-width': [1.0, 1.0],
        'petal-length': [1.0, 1.0],
        'petal-width': [1.0, 1.0]
    })
)

Exception: Request failed with status code 400: Cannot make prediction: could not convert string to float: 'WRONG value'